In [13]:
%cd PRO503

[Errno 2] No such file or directory: 'PRO503'
/workspace/PRO503


/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType, FloatType,BooleanType


spark = ( SparkSession.builder.appName("PRO503").master("spark://spark-master:7077").getOrCreate())

sc = spark.sparkContext

In [15]:
main_schema = StructType([
    StructField("Crop",StringType(),True),
    StructField("Region",StringType(),True),
    StructField("Soil_Type",StringType(),True),
    StructField("Soil_pH",FloatType(),True),
    StructField("Rainfall_mm",FloatType(),True),
    StructField("Temperature_C",FloatType(),True),
    StructField("Humidity_pct",FloatType(),True),
    StructField("Fertilizer_Used_Kg",FloatType(),True),
    StructField("Irrigation",StringType(),True),
    StructField("Pesticides_Used_kg",FloatType(),True),
    StructField("Planting_Density",FloatType(),True),
    StructField("Previus_Crop",StringType(),True),
    StructField("Yield_ton_per_ha",FloatType(),True)
])

df_crop = (spark.read.format("csv")
      .option("header","true")
      .schema(main_schema)
      .load("crop_yield_dataset.csv"))



In [16]:
from pyspark.sql import functions as fun

df_eng = df_crop.withColumn(
    "Crop_ID",
    fun.concat(
        fun.lit("CODIGO_"),
        fun.lpad(fun.split(fun.col("Region"), "_").getItem(1), 3, "X"),
        fun.lit("-"),
        fun.upper(fun.col("Crop"))
    )
)

df_eng.select("Region", "Crop", "Crop_ID").show(25, truncate=False)

+--------+------+-----------------+
|Region  |Crop  |Crop_ID          |
+--------+------+-----------------+
|Region_C|Maize |CODIGO_XXC-MAIZE |
|Region_D|Barley|CODIGO_XXD-BARLEY|
|Region_C|Rice  |CODIGO_XXC-RICE  |
|Region_D|Maize |CODIGO_XXD-MAIZE |
|Region_D|Maize |CODIGO_XXD-MAIZE |
|Region_C|Barley|CODIGO_XXC-BARLEY|
|Region_B|Rice  |CODIGO_XXB-RICE  |
|Region_D|Rice  |CODIGO_XXD-RICE  |
|Region_C|Maize |CODIGO_XXC-MAIZE |
|Region_A|Wheat |CODIGO_XXA-WHEAT |
|Region_C|Maize |CODIGO_XXC-MAIZE |
|Region_A|Maize |CODIGO_XXA-MAIZE |
|Region_D|Maize |CODIGO_XXD-MAIZE |
|Region_C|Maize |CODIGO_XXC-MAIZE |
|Region_C|Barley|CODIGO_XXC-BARLEY|
|Region_A|Rice  |CODIGO_XXA-RICE  |
|Region_C|Barley|CODIGO_XXC-BARLEY|
|Region_C|Barley|CODIGO_XXC-BARLEY|
|Region_D|Barley|CODIGO_XXD-BARLEY|
|Region_C|Maize |CODIGO_XXC-MAIZE |
|Region_B|Wheat |CODIGO_XXB-WHEAT |
|Region_A|Rice  |CODIGO_XXA-RICE  |
|Region_C|Wheat |CODIGO_XXC-WHEAT |
|Region_A|Barley|CODIGO_XXA-BARLEY|
|Region_A|Barley|CODIGO_XXA-

In [17]:
df_eng = df_eng.withColumn(
    "Log_Rainfall", 
    fun.log(fun.col("Rainfall_mm") + 1)
).withColumn(
    "Yield_Redondeado", 
    fun.round(fun.col("Yield_ton_per_ha"), 1)
).withColumn(
    "Rendimiento_Bancario", 
    fun.bround(fun.col("Yield_ton_per_ha"), 0)
)

In [18]:
df_eng = df_eng.withColumn(
    "Max_Quimico_kg", 
    fun.greatest(fun.col("Fertilizer_Used_Kg"), fun.col("Pesticides_Used_kg"))
)

df_eng.select(
    "Fertilizer_Used_Kg", 
    "Pesticides_Used_kg", 
    "Max_Quimico_kg"
).show(5)

+------------------+------------------+--------------+
|Fertilizer_Used_Kg|Pesticides_Used_kg|Max_Quimico_kg|
+------------------+------------------+--------------+
|             105.1|              10.2|         105.1|
|             221.8|              35.5|         221.8|
|              61.2|              40.0|          61.2|
|             257.8|              42.7|         257.8|
|             195.8|              25.5|         195.8|
+------------------+------------------+--------------+
only showing top 5 rows



In [19]:
df_eng = df_eng.withColumn(
    "Fecha_Siembra", 
    fun.to_date(fun.lit("2023-04-01"))
).withColumn(
    "Fecha_Estimada_Cosecha", 
    fun.date_add(fun.col("Fecha_Siembra"), 150)
).withColumn(
    "Mes_Cosecha", 
    fun.month(fun.col("Fecha_Estimada_Cosecha"))
)

df_eng.select("Fecha_Siembra", "Fecha_Estimada_Cosecha", "Mes_Cosecha").show(5)

+-------------+----------------------+-----------+
|Fecha_Siembra|Fecha_Estimada_Cosecha|Mes_Cosecha|
+-------------+----------------------+-----------+
|   2023-04-01|            2023-08-29|          8|
|   2023-04-01|            2023-08-29|          8|
|   2023-04-01|            2023-08-29|          8|
|   2023-04-01|            2023-08-29|          8|
|   2023-04-01|            2023-08-29|          8|
+-------------+----------------------+-----------+
only showing top 5 rows

